### Word Group In Youtube Firebase

In [1]:
import firebase_admin
from firebase_admin import credentials, firestore, storage

In [2]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [3]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [4]:
# language pair
lang_folder = "French"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# language detail num
if lang_folder == "English":
    lang_id = "1"
elif lang_folder == "Turkish":
    lang_id = "2"
elif lang_folder == "French":
    lang_id = "3"
elif lang_folder == "Spanish":
    lang_id = "4"
elif lang_folder == "Portuguese":
    lang_id = "5"
elif lang_folder == "Dutch":
    lang_id = "6"
elif lang_folder == "Italian":
    lang_id = "7"
elif lang_folder == "German":
    lang_id = "8"
elif lang_folder == "Arabic":
    lang_id = "9"
else:
    pass

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"Language = {lang_folder}\nLang Id = {lang_id}\nFile Extention = {file_ext}")

Language = French
Lang Id = 3
File Extention = 6


In [5]:
path = f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Language Level/Result/3-Word Group In Youtube Sentence Firebase"

#Path(path).mkdir(parents=True, exist_ok=True)

In [6]:
#cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Flash Card/Api Key/flashcard-1c46e-firebase-adminsdk-b4kn5-5ddbde56b3.json")  # Flash Card
cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Language Level/Api Key/language-levels-firebase-adminsdk-u1m3n-87de34db73.json")  # Language Level
app = firebase_admin.initialize_app(cred)

In [7]:
fire_db = firestore.client()

#### Read Data

In [8]:
#parent_folder_path = f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Language Level/Data/1-Word Group In Youtube Sentence"
parent_folder_path = f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Language Level/Result/2-Word Group Prepare For Youtube Tool"

In [9]:
df_28 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.csv")
df_28 = df_28.head(200)
df_28

,search_string,start_time,end_time,sentence,video_id,video_url
0,mais cest pas pour ça que ça,1180,1182,mais cest pas pour ça que ça va arriver,YuxMdNbmxzM,https://www.youtube.com/watch?v=YuxMdNbmxzM&t=...
1,cest pour ça que je vous le,672,675,cest pour ça que je vous le montre,aDBLqp0bIPw,https://www.youtube.com/watch?v=aDBLqp0bIPw&t=...
2,cest pour ça que je vous le,261,262,cest pour ça que je vous le dis,NuBCoiQLhHY,https://www.youtube.com/watch?v=NuBCoiQLhHY&t=...
3,et cest pour ça que dans ce,3402,3405,et cest pour ça que dans ce forum,GWSsZ6KCuJI,https://www.youtube.com/watch?v=GWSsZ6KCuJI&t=...
4,cest pour ça que jai pas de,926,929,cest pour ça que jai pas de mec,KAgyTxbTNxw,https://www.youtube.com/watch?v=KAgyTxbTNxw&t=...
...,...,...,...,...,...,...
195,pour vous les,30,34,pour vous les,RdwCEp5kkDM,https://www.youtube.com/watch?v=RdwCEp5kkDM&t=30s
196,pour vous les,35,38,pour vous les,4s2TR89BoAc,https://www.youtube.com/watch?v=4s2TR89BoAc&t=35s
197,pour en et en,1016,1028,pour en et en non,1_fiSHLNaLI,https://www.youtube.com/watch?v=1_fiSHLNaLI&t=...
198,mais pour les,187,191,mais pour les,ulqpPjXmXiA,https://www.youtube.com/watch?v=ulqpPjXmXiA&t=...


In [10]:
df_200 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.csv")
df_200 = df_200.head(200)
df_200

,search_string,start_time,end_time,sentence,video_id,video_url
0,dans la vie quand tout va bien mais aussi quan...,618,624,dans la vie quand tout va bien mais aussi quan...,8ywehhHHhrU,https://www.youtube.com/watch?v=8ywehhHHhrU&t=...
1,je lui dis que je vais bien que voilà cest la ...,951,960,moment je lui dis que je vais bien que voilà c...,6NIUzeuw6Lk,https://www.youtube.com/watch?v=6NIUzeuw6Lk&t=...
2,donc là ce avait un peu de temps je sais pas p...,775,779,donc là ce avait un peu de temps je sais pas p...,4YJSOd91bz4,https://www.youtube.com/watch?v=4YJSOd91bz4&t=...
3,quand même quoi donc parce que voilà cest fait...,157,163,quand même quoi donc parce que voilà cest fait...,0_sv895JPlc,https://www.youtube.com/watch?v=0_sv895JPlc&t=...
4,là tu ne fais pas pour toi tu le fais parce qu...,159,163,là tu ne fais pas pour toi tu le fais parce qu...,qTvOYQ_HMCA,https://www.youtube.com/watch?v=qTvOYQ_HMCA&t=...
...,...,...,...,...,...,...
195,maintenant quand tu veux merci ok on va donc m...,121,125,maintenant quand tu veux merci ok on va donc m...,7Ijp_mrOhkw,https://www.youtube.com/watch?v=7Ijp_mrOhkw&t=...
196,mais bon tout le monde veut quand même se dire...,281,284,mais bon tout le monde veut quand même se dire...,lsxGXj264zs,https://www.youtube.com/watch?v=lsxGXj264zs&t=...
197,et après ça questce qui sest passé et bien tu ...,227,230,et après ça questce qui sest passé et bien tu ...,z6YuIQan1rg,https://www.youtube.com/watch?v=z6YuIQan1rg&t=...
198,alors vous allez me dire ok mais comment on fa...,275,279,alors vous allez me dire ok mais comment on fa...,5VXZ53DrIpQ,https://www.youtube.com/watch?v=5VXZ53DrIpQ&t=...


In [11]:
df_1000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.csv")
df_1000 = df_1000.head(200)
df_1000

,search_string,start_time,end_time,sentence,video_id,video_url
0,pour le coup si tu comprends pas que tu peux p...,386,391,pour le coup si tu comprends pas que tu peux p...,mEwv2MslK10,https://www.youtube.com/watch?v=mEwv2MslK10&t=...
1,voilà oui on pourrait passer encore beaucoup p...,5913,5920,voilà oui on pourrait passer encore beaucoup p...,Tndgvn3FoZU,https://www.youtube.com/watch?v=Tndgvn3FoZU&t=...
2,bon moi je travaille tous les jours aujourdhui...,3757,3763,bon moi je travaille tous les jours aujourdhui...,v0oH5e92pO0,https://www.youtube.com/watch?v=v0oH5e92pO0&t=...
3,cest que cest pas pour tout de suite parce quà...,732,738,lidée cest que cest pas pour tout de suite par...,KZPJU5ggkJk,https://www.youtube.com/watch?v=KZPJU5ggkJk&t=...
4,la question je lui dis mais dis moi tu pourrai...,1397,1403,du cimetière et je lui pose la question je lui...,lMUZkh7Cbwc,https://www.youtube.com/watch?v=lMUZkh7Cbwc&t=...
...,...,...,...,...,...,...
195,alors cest super simple vraiment simple et on ...,41,48,alors cest super simple vraiment simple et on ...,JFKtSI3Ph58,https://www.youtube.com/watch?v=JFKtSI3Ph58&t=41s
196,ce soit dautre donc tu as le choix si tu veux ...,736,742,ce soit dautre donc tu as le choix si tu veux ...,jdrNadJWLmY,https://www.youtube.com/watch?v=jdrNadJWLmY&t=...
197,quil faut faire bien attention où vous voulez ...,32,38,quil faut faire bien attention où vous voulez ...,NoIflfkccv4,https://www.youtube.com/watch?v=NoIflfkccv4&t=32s
198,te fait gagner un point pour savoir cette pers...,91,98,te fait gagner un point pour savoir cette pers...,fdN0gNXHw5U,https://www.youtube.com/watch?v=fdN0gNXHw5U&t=91s


In [12]:
df_5000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.csv")
df_5000 = df_5000.head(200)
df_5000

,search_string,start_time,end_time,sentence,video_id,video_url
0,ah aussi je voulais vous dire je vais faire un...,314,321,ah aussi je voulais vous dire je vais faire un...,oig_hpR47OM,https://www.youtube.com/watch?v=oig_hpR47OM&t=...
1,femme mais non cest pas grave écoute de un ell...,238,245,femme mais non cest pas grave écoute de un ell...,sH7T97l3Bvo,https://www.youtube.com/watch?v=sH7T97l3Bvo&t=...
2,faire le point puisque nous sommes bientôt fin...,14,26,faire le point puisque nous sommes bientôt fin...,8Kn1XH8MHsQ,https://www.youtube.com/watch?v=8Kn1XH8MHsQ&t=14s
3,et nous on se retrouve très très vite pour une...,589,595,et nous on se retrouve très très vite pour une...,Y862SCIlOto,https://www.youtube.com/watch?v=Y862SCIlOto&t=...
4,ça jai pas de questions làdessus en revanche c...,985,997,ça jai pas de questions làdessus en revanche c...,5YPMBr4Nngg,https://www.youtube.com/watch?v=5YPMBr4Nngg&t=...
...,...,...,...,...,...,...
195,pourquoi estce quon se voit cest pour savoir p...,1006,1011,pourquoi estce quon se voit cest pour savoir p...,VEdhi7xsV0o,https://www.youtube.com/watch?v=VEdhi7xsV0o&t=...
196,si cest elle ce soir vous vous dites donc quel...,628,632,si cest elle ce soir vous vous dites donc quel...,ON4k3iaTOk8,https://www.youtube.com/watch?v=ON4k3iaTOk8&t=...
197,nous avons peur ceux aussi qui nous mettent en...,112,122,nous avons peur ceux aussi qui nous mettent en...,ghyBT626p1o,https://www.youtube.com/watch?v=ghyBT626p1o&t=...
198,mais plutôt pour vous raconter un petit peu vo...,302,306,mais plutôt pour vous raconter un petit peu vo...,pG7qWAC96Q4,https://www.youtube.com/watch?v=pG7qWAC96Q4&t=...


In [13]:
df_10000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.csv")
df_10000 = df_10000.head(200)
df_10000

,search_string,start_time,end_time,sentence,video_id,video_url
0,mais au final je dois être honnête et dire que...,413,435,mais au final je dois être honnête et dire que...,9g1rjIjSUW0,https://www.youtube.com/watch?v=9g1rjIjSUW0&t=...
1,passé des moments heureux dans cette maison av...,452,467,jean a passé des moments heureux dans cette ma...,YypPgUlKJt4,https://www.youtube.com/watch?v=YypPgUlKJt4&t=...
2,ah aussi je voulais vous dire je vais faire un...,314,321,ah aussi je voulais vous dire je vais faire un...,oig_hpR47OM,https://www.youtube.com/watch?v=oig_hpR47OM&t=...
3,un coeur très différent et on ne peut pas amen...,720,733,un coeur très différent et on ne peut pas amen...,h6VaYkFE4yw,https://www.youtube.com/watch?v=h6VaYkFE4yw&t=...
4,même si le meilleur moyen de réduire nos déche...,345,354,même si le meilleur moyen de réduire nos déche...,eXE0dzAXErg,https://www.youtube.com/watch?v=eXE0dzAXErg&t=...
...,...,...,...,...,...,...
195,ont totalement changé également il est désorma...,1060,1065,ses live ont totalement changé également il es...,crHZYY9cmDE,https://www.youtube.com/watch?v=crHZYY9cmDE&t=...
196,sauf peutêtre pour ses grands yeux rouges qui ...,166,170,sauf peutêtre pour ses grands yeux rouges qui ...,acID9P7_V1k,https://www.youtube.com/watch?v=acID9P7_V1k&t=...
197,en faite chacun construit son panneau stop le ...,865,873,en faite chacun construit son panneau stop le ...,lYwbO55v7TU,https://www.youtube.com/watch?v=lYwbO55v7TU&t=...
198,ou qui en ont besoin ils ont mis en place une ...,750,757,ou qui en ont besoin ils ont mis en place une ...,1PpCs5sAFqY,https://www.youtube.com/watch?v=1PpCs5sAFqY&t=...


In [14]:
#df_20000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.csv")
df_20000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.xlsx")
df_20000 = df_20000.head(200)
df_20000

,search_string,start_time,end_time,sentence,video_id,video_url
0,même si le meilleur moyen de réduire nos déche...,345,354,même si le meilleur moyen de réduire nos déche...,eXE0dzAXErg,https://www.youtube.com/watch?v=eXE0dzAXErg&t=...
1,faire le point puisque nous sommes bientôt fin...,14,26,faire le point puisque nous sommes bientôt fin...,8Kn1XH8MHsQ,https://www.youtube.com/watch?v=8Kn1XH8MHsQ&t=14s
2,javais envie de javais besoin dexprimer au tra...,326,337,javais envie de javais besoin dexprimer au tra...,Cr9pam0A_O0,https://www.youtube.com/watch?v=Cr9pam0A_O0&t=...
3,et nous voilà enfin devant théo pour la finale...,1183,1190,et nous voilà enfin devant théo pour la finale...,JwbVn3tYo8s,https://www.youtube.com/watch?v=JwbVn3tYo8s&t=...
4,cest tout ce que valent ces souvenirs cette po...,418,435,cest tout ce que valent ces souvenirs cette po...,YypPgUlKJt4,https://www.youtube.com/watch?v=YypPgUlKJt4&t=...
...,...,...,...,...,...,...
195,il ne faut pas imaginer que ça les arrange je ...,1862,1870,il ne faut pas imaginer que ça les arrange je ...,ZnqFzjekTyc,https://www.youtube.com/watch?v=ZnqFzjekTyc&t=...
196,pour linstant qui nest pas super bien rangé do...,969,977,pour linstant qui nest pas super bien rangé do...,8c09RTp_Ieg,https://www.youtube.com/watch?v=8c09RTp_Ieg&t=...
197,il est tombé amoureux ces derniers mois dune j...,342,351,il est tombé amoureux ces derniers mois dune j...,SuxXcI4ZpG4,https://www.youtube.com/watch?v=SuxXcI4ZpG4&t=...
198,on se dit ah mais tiens effectivement maintena...,2384,2392,on se dit ah mais tiens effectivement maintena...,_PMmvDg9gaA,https://www.youtube.com/watch?v=_PMmvDg9gaA&t=...


In [15]:
#df_40000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.csv")
df_40000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.xlsx")
df_40000 = df_40000.head(200)
df_40000

,search_string,start_time,end_time,sentence,video_id,video_url
0,ils ne respectent pas toujours la réglementati...,353,371,ils ne respectent pas toujours la réglementati...,fcdtNH07Ip0,https://www.youtube.com/watch?v=fcdtNH07Ip0&t=...
1,arriver mathieu ne néglige rien il vise lexcel...,642,662,et pour y arriver mathieu ne néglige rien il v...,OGkVgSZI8f8,https://www.youtube.com/watch?v=OGkVgSZI8f8&t=...
2,mais comme toujours ce matériau nest pas parfa...,365,377,mais comme toujours ce matériau nest pas parfa...,e2ITAibdtWo,https://www.youtube.com/watch?v=e2ITAibdtWo&t=...
3,jai mis dautres choses en place de lhypnose de...,52,62,jai mis dautres choses en place de lhypnose de...,2T4jG33szys,https://www.youtube.com/watch?v=2T4jG33szys&t=52s
4,tout le long du projet il a participé en poste...,290,303,tout le long du projet il a participé en poste...,9g1rjIjSUW0,https://www.youtube.com/watch?v=9g1rjIjSUW0&t=...
...,...,...,...,...,...,...
195,mais nous petits joueurs deurope tu crois quon...,224,229,mais nous petits joueurs deurope tu crois quon...,2uRb0IsQTnw,https://www.youtube.com/watch?v=2uRb0IsQTnw&t=...
196,on na limpression quon nous offre un cadeau la...,53,60,on na limpression quon nous offre un cadeau la...,uXYnEV9atxg,https://www.youtube.com/watch?v=uXYnEV9atxg&t=53s
197,larguer les trois moteurs principaux ne sont p...,541,548,larguer les trois moteurs principaux ne sont p...,xAbEMo7Tn64,https://www.youtube.com/watch?v=xAbEMo7Tn64&t=...
198,les virus sont installés en fait je répète par...,1405,1412,les virus sont installés en fait je répète par...,Tndgvn3FoZU,https://www.youtube.com/watch?v=Tndgvn3FoZU&t=...


#### Sending Data To Firestore

##### Language Data

In [16]:
language_data = {"languages":[{"id":1, "name":"English"},{"id":2, "name":"Turkish"},{"id":3, "name":"French"},
                              {"id":4,"name":"Spanish"},{"id":5, "name":"Portuguese"},{"id":6, "name":"Dutch"},
                              {"id":7, "name":"Italian"},{"id":8, "name":"German"},{"id":9, "name":"Arabic"}]}

In [17]:
#fire_db.collection("language-data").document("languages").set(language_data)

##### Language Detail

##### Absolute Beginner

In [18]:
# Absolute Beginner Data
end_time_list = df_28["end_time"].to_list()
video_id_list = df_28["video_id"].to_list()
start_time_list = df_28["start_time"].to_list()

In [19]:
firestore_beginner_data = zip(end_time_list,video_id_list,start_time_list)
beginner_description_text = "description"

In [20]:
beginner_data_list = []
for endtime, videoid, starttime in firestore_beginner_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    beginner_data_list.append(data)
    
#beginner_data_list   

In [21]:
data_beginner = {"description":f"{beginner_description_text}", "level":0, "levelName":"Absolute Beginner", "videos":beginner_data_list}
#data_beginner

In [22]:
#fire_db.collection("language-detail-data").document(f"{lang_id}").set(data_beginner)

##### A1 Level

In [23]:
# A1 Level Data
end_time_list = df_200["end_time"].to_list()
video_id_list = df_200["video_id"].to_list()
start_time_list = df_200["start_time"].to_list()

In [24]:
firestore_A1_data = zip(end_time_list,video_id_list,start_time_list)
A1_description_text = "A1 level description"

In [25]:
A1_data_list = []
for endtime, videoid, starttime in firestore_A1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    A1_data_list.append(data)

#A1_data_list   

In [26]:
data_A1 = {"description":f"{A1_description_text}", "level":1, "levelName":"A1", "videos":A1_data_list}
#data_A1

##### A2 Level

In [27]:
# A2 Level Data
end_time_list = df_1000["end_time"].to_list()
video_id_list = df_1000["video_id"].to_list()
start_time_list = df_1000["start_time"].to_list()

In [28]:
firestore_A2_data = zip(end_time_list,video_id_list,start_time_list)
A2_description_text = "A2 level description"

In [29]:
A2_data_list = []
for endtime, videoid, starttime in firestore_A2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    A2_data_list.append(data)

#A2_data_list   

In [30]:
data_A2 = {"description":f"{A2_description_text}", "level":2, "levelName":"A2", "videos":A2_data_list}
#data_A2

##### B1 Level

In [31]:
# B1 Level Data
end_time_list = df_5000["end_time"].to_list()
video_id_list = df_5000["video_id"].to_list()
start_time_list = df_5000["start_time"].to_list()

In [32]:
firestore_B1_data = zip(end_time_list,video_id_list,start_time_list)
B1_description_text = "B1 level description"

In [33]:
B1_data_list = []
for endtime, videoid, starttime in firestore_B1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    B1_data_list.append(data)

#B1_data_list   

In [34]:
data_B1 = {"description":f"{B1_description_text}", "level":3, "levelName":"B1", "videos":B1_data_list}
#data_B1

##### B2 Level

In [35]:
# B2 Level Data
end_time_list = df_10000["end_time"].to_list()
video_id_list = df_10000["video_id"].to_list()
start_time_list = df_10000["start_time"].to_list()

In [36]:
firestore_B2_data = zip(end_time_list,video_id_list,start_time_list)
B2_description_text = "B2 level description"

In [37]:
B2_data_list = []
for endtime, videoid, starttime in firestore_B2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    B2_data_list.append(data)

#B2_data_list   

In [38]:
data_B2 = {"description":f"{B2_description_text}", "level":4, "levelName":"B2", "videos":B2_data_list}
#data_B2

##### C1 Level

In [39]:
# C1 Level Data
video_id_list = df_20000["video_id"].to_list()
end_time_list = df_20000["end_time"].to_list()
start_time_list = df_20000["start_time"].to_list()

In [40]:
firestore_C1_data = zip(end_time_list,video_id_list,start_time_list)
C1_description_text = "C1 level description"

In [41]:
C1_data_list = []
for endtime, videoid, starttime in firestore_C1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    C1_data_list.append(data)

#C1_data_list   

In [42]:
data_C1 = {"description":f"{C1_description_text}", "level":5, "levelName":"C1", "videos":C1_data_list}
#data_C1

##### C2 Level

In [43]:
# C2 Level Data
video_id_list = df_40000["video_id"].to_list()
end_time_list = df_40000["end_time"].to_list()
start_time_list = df_40000["start_time"].to_list()

In [44]:
firestore_C2_data = zip(end_time_list,video_id_list,start_time_list)
C2_description_text = "C2 level description"

In [45]:
C2_data_list = []
for endtime, videoid, starttime in firestore_C2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    C2_data_list.append(data)

#C2_data_list   

In [46]:
data_C2 = {"description":f"{C2_description_text}", "level":6, "levelName":"C2", "videos":C2_data_list}
#data_C2

##### All Data Sending

In [47]:
data_all = {"datas":[data_beginner, data_A1, data_A2, data_B1, data_B2, data_C1, data_C2]}
#data_all

In [48]:
fire_db.collection("language-detail-data").document(f"{lang_id}").set(data_all)

update_time {
  seconds: 1674122178
  nanos: 673978000
}

#### Copy Move And Delete

In [ ]:
output_file2 = glob.glob(f"{lang_folder.capitalize()}_Word_Group*{file_ext}.xlsx")
output_file2

In [ ]:
for l in output_file2:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for j in output_file2:
    try:
        os.remove(j)
    except:
        pass